In [28]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from math import factorial
import numpy as np
from scipy import stats as st

In [29]:
visitas = pd.read_csv('datos/visits_log_us.csv')

In [30]:
pedidos = pd.read_csv('datos/orders_log_us.csv')

In [31]:
gastos = pd.read_csv('datos/costs_us.csv')

VISITAS 

In [32]:
visitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB


In [33]:
visitas.head()

,Device,End Ts,Source Id,Start Ts,Uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168


In [34]:
porcentaje_nulos = visitas.isnull().mean() * 100
print(porcentaje_nulos)

Device       0.0
End Ts       0.0
Source Id    0.0
Start Ts     0.0
Uid          0.0
dtype: float64


In [35]:
visitas = visitas.drop_duplicates()
visitas = visitas.reset_index(drop=True)


In [36]:
visitas.head()

,Device,End Ts,Source Id,Start Ts,Uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168


Convertiremos las columnas 'Start Ts' y 'End Ts' a formato datetime para poder trabajar con ellas

In [37]:
visitas['Start Ts'] = pd.to_datetime(visitas['Start Ts'])
visitas['End Ts'] = pd.to_datetime(visitas['End Ts'])
print(visitas[['Start Ts', 'End Ts']].dtypes)


Start Ts    datetime64[ns]
End Ts      datetime64[ns]
dtype: object


Ahora Trabajaremos para encontrar la cantidad de usuarios por día, semana, mes

In [38]:
u_dia = visitas.groupby(visitas['Start Ts'].dt.date)['Uid'].nunique()
u_semana = visitas.groupby(visitas['Start Ts'].dt.to_period('W'))['Uid'].nunique()
u_mes = visitas.groupby(visitas['Start Ts'].dt.to_period('M'))['Uid'].nunique()

print(u_dia.head(5))
print(u_semana.head(5))
print(u_mes.head(5))

Start Ts
2017-06-01    605
2017-06-02    608
2017-06-03    445
2017-06-04    476
2017-06-05    820
Name: Uid, dtype: int64
Start Ts
2017-05-29/2017-06-04    2021
2017-06-05/2017-06-11    4129
2017-06-12/2017-06-18    2812
2017-06-19/2017-06-25    2878
2017-06-26/2017-07-02    3064
Freq: W-SUN, Name: Uid, dtype: int64
Start Ts
2017-06    13259
2017-07    14183
2017-08    11631
2017-09    18975
2017-10    29692
Freq: M, Name: Uid, dtype: int64


Revisaremos cuantas sesiones se realizan por día.

In [39]:
ses_por_dia = visitas.groupby(visitas['Start Ts'].dt.date)['Uid'].count()
print("Sesiones por dia:\n", ses_por_dia.head(5))

Sesiones por dia:
 Start Ts
2017-06-01    664
2017-06-02    658
2017-06-03    477
2017-06-04    510
2017-06-05    893
Name: Uid, dtype: int64


Revisaremos si un usuario ingresa más de una vez

In [40]:
ses_por_us = visitas.groupby('Uid')['Start Ts'].count()
multiple_ses_us = ses_por_us[ses_por_us > 1]

print("Usuarios con más de una Sesios por Día:\n", multiple_ses_us)


Usuarios con más de una Sesios por Día:
 Uid
313578113262317         3
325320750514679         2
526778907996220         4
577434573913691         2
1260655184775459        4
                       ..
18445884613277162497    3
18446104389491815722    2
18446156210226471712    7
18446167067214817906    2
18446621818809592527    2
Name: Start Ts, Length: 52128, dtype: int64


Vamos a revisar cuanto dura cada sesión del visitante

In [41]:
visitas['duracion'] = visitas['End Ts'] - visitas['Start Ts']
print("Duracion de Cada Sesion:\n", visitas['duracion'])

Duracion de Cada Sesion:
 0        0 days 00:18:00
1        0 days 00:28:00
2        0 days 00:00:00
3        0 days 00:24:00
4        0 days 00:00:00
               ...      
359395   0 days 00:00:19
359396   0 days 00:00:19
359397   0 days 00:00:19
359398   0 days 00:00:19
359399   0 days 00:00:19
Name: duracion, Length: 359400, dtype: timedelta64[ns]


Revisaremos con que frecuencia regresa al sitio cada usuario

In [42]:

visitas = visitas.sort_values(by=['Uid','Start Ts'])
frecuencia_regreso = visitas.groupby('Uid')['Start Ts'].diff()
print(frecuencia_regreso)

196543                 NaT
257333                 NaT
173818                 NaT
173142                 NaT
192554   152 days 16:28:00
                ...       
175259                 NaT
293449                 NaT
319162                 NaT
222173   144 days 20:03:00
132805                 NaT
Name: Start Ts, Length: 359400, dtype: timedelta64[ns]


PEDIDOS

In [43]:
pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB


In [44]:
pedidos.head(5)

,Buy Ts,Revenue,Uid
0,2017-06-01 00:10:00,17.00,10329302124590727494
1,2017-06-01 00:25:00,0.55,11627257723692907447
2,2017-06-01 00:27:00,0.37,17903680561304213844
3,2017-06-01 00:29:00,0.55,16109239769442553005
4,2017-06-01 07:58:00,0.37,14200605875248379450


In [45]:
print(pedidos.isnull().sum())

Buy Ts     0
Revenue    0
Uid        0
dtype: int64


In [46]:
pedidos = pedidos.drop_duplicates()
pedidos = pedidos.reset_index(drop=True)

Convertiremos la columna 'Buy Ts' a formato datetime para poder trabajar con ella

In [47]:
pedidos['Buy Ts'] = pd.to_datetime(pedidos['Buy Ts'])
print(pedidos['Buy Ts'].dtypes)


datetime64[ns]


Vamos a obtener la primera Orden

In [48]:
primera_orden = pedidos.groupby('Uid')['Buy Ts'].min()
print(primera_orden.head())

Uid
313578113262317    2018-01-03 21:51:00
1575281904278712   2017-06-03 10:13:00
2429014661409475   2017-10-11 18:33:00
2464366381792757   2018-01-28 15:54:00
2551852515556206   2017-11-24 10:14:00
Name: Buy Ts, dtype: datetime64[ns]


Vamos a obtener la primera compra

In [49]:
primera_orden = primera_orden.to_frame(name='primera_compra')
pedidos = pedidos.merge(primera_orden, left_on='Uid', right_index=True, how='left')
print(pedidos.head())

               Buy Ts  Revenue                   Uid      primera_compra
0 2017-06-01 00:10:00    17.00  10329302124590727494 2017-06-01 00:10:00
1 2017-06-01 00:25:00     0.55  11627257723692907447 2017-06-01 00:25:00
2 2017-06-01 00:27:00     0.37  17903680561304213844 2017-06-01 00:27:00
3 2017-06-01 00:29:00     0.55  16109239769442553005 2017-06-01 00:29:00
4 2017-06-01 07:58:00     0.37  14200605875248379450 2017-06-01 07:58:00


Vamos a trabajar para obtener los cohortes Mensual

In [50]:
pedidos['primera_compra'] = pd.to_datetime(pedidos['primera_compra'], errors='coerce')
pedidos['Buy Ts'] = pd.to_datetime(pedidos['Buy Ts'], errors='coerce')
pedidos['primera_orden_mes'] = pedidos['primera_compra'].dt.to_period('M')
pedidos['orden_mes'] = pedidos['Buy Ts'].dt.to_period('M')
print(pedidos.head())

               Buy Ts  Revenue                   Uid      primera_compra  \
0 2017-06-01 00:10:00    17.00  10329302124590727494 2017-06-01 00:10:00   
1 2017-06-01 00:25:00     0.55  11627257723692907447 2017-06-01 00:25:00   
2 2017-06-01 00:27:00     0.37  17903680561304213844 2017-06-01 00:27:00   
3 2017-06-01 00:29:00     0.55  16109239769442553005 2017-06-01 00:29:00   
4 2017-06-01 07:58:00     0.37  14200605875248379450 2017-06-01 07:58:00   

  primera_orden_mes orden_mes  
0           2017-06   2017-06  
1           2017-06   2017-06  
2           2017-06   2017-06  
3           2017-06   2017-06  
4           2017-06   2017-06  


Analizaremos las compras mensuales de cada Cohort

In [51]:
grupo_cohort = pedidos.groupby('primera_orden_mes').agg({'Uid': 'nunique', 'Revenue': 'sum'})
print(grupo_cohort.head())

                    Uid   Revenue
primera_orden_mes                
2017-06            2023  24031.69
2017-07            1923  16127.92
2017-08            1370  11606.26
2017-09            2581  34676.32
2017-10            4340  27603.45


Podemos observar que los ingresos por cohortes bajaron, se recuperaron pero nuevamente bajaron y otra vez hay un repunte

Realizaremos una tabla Dinámica para conocer que clientes siguen realizando compras despues de que realizaron la primera

In [52]:
pedidos.pivot_table(
    index='primera_orden_mes',
    columns='orden_mes',
    values='Uid',
    aggfunc='nunique',)

orden_mes,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06
primera_orden_mes,,,,,,,,,,,,,
2017-06,2023.0,61.0,50.0,54.0,88.0,67.0,62.0,47.0,58.0,45.0,45.0,53.0,NaN
2017-07,NaN,1923.0,52.0,57.0,64.0,49.0,38.0,36.0,39.0,42.0,22.0,26.0,NaN
2017-08,NaN,NaN,1370.0,58.0,53.0,44.0,40.0,32.0,30.0,44.0,19.0,31.0,NaN
2017-09,NaN,NaN,NaN,2581.0,130.0,100.0,74.0,52.0,64.0,66.0,37.0,43.0,NaN
2017-10,NaN,NaN,NaN,NaN,4340.0,206.0,123.0,92.0,93.0,72.0,56.0,67.0,NaN
2017-11,NaN,NaN,NaN,NaN,NaN,4081.0,222.0,120.0,106.0,81.0,48.0,62.0,NaN
2017-12,NaN,NaN,NaN,NaN,NaN,NaN,4383.0,146.0,103.0,97.0,50.0,63.0,NaN
2018-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3373.0,114.0,83.0,43.0,45.0,NaN
2018-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3651.0,118.0,58.0,39.0,NaN


Haremos un Gráfico de Lineas


Existe una disminución de clientes despues del primer mes, luego se recupera y nuevamente hay una caida y otra vez se recupera.

Haremos un gráfico de lineas

Vamos a calcular el promedio de compra(ingresos)

In [53]:
pedidos_agrupados_cohorts = pedidos.groupby( ['primera_orden_mes', 'orden_mes']).agg({'Revenue': 'sum', 'Uid': 'nunique'})
print(pedidos_agrupados_cohorts.head())

                             Revenue   Uid
primera_orden_mes orden_mes               
2017-06           2017-06    9557.49  2023
                  2017-07     981.82    61
                  2017-08     885.34    50
                  2017-09    1931.30    54
                  2017-10    2068.58    88


In [54]:
pedidos_agrupados_cohorts.columns

Index(['Revenue', 'Uid'], dtype='object')

Buscaremos la compra por usuario

In [55]:
pedidos_agrupados_cohorts['Ingreso_por_uid'] = (pedidos_agrupados_cohorts['Revenue']
    / pedidos_agrupados_cohorts['Uid'])

Realizaremos una Tabla Dinámica para observar la compra mensual por usuario.

In [56]:
pedidos_agrupados_cohorts.pivot_table(
    index='primera_orden_mes',
    columns='orden_mes',
    values='Ingreso_por_uid',
    aggfunc='mean',)

orden_mes,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06
primera_orden_mes,,,,,,,,,,,,,
2017-06,4.724414,16.095410,17.706800,35.764815,23.506591,22.207761,31.011935,25.033191,19.295690,27.233556,25.681333,9.804151,NaN
2017-07,NaN,6.010218,12.396346,21.035965,10.786094,6.938163,7.896842,6.421111,6.992821,7.382143,12.861818,11.513846,NaN
2017-08,NaN,NaN,5.276518,11.148793,11.851321,12.182955,16.921250,12.139062,9.620333,12.610455,21.070000,8.307419,NaN
2017-09,NaN,NaN,NaN,5.644529,22.188385,13.445200,138.669189,19.881538,26.095000,27.437121,16.961351,11.044651,NaN
2017-10,NaN,NaN,NaN,NaN,5.003733,11.287427,6.753252,7.413152,7.072796,7.255139,6.573214,7.479701,NaN
2017-11,NaN,NaN,NaN,NaN,NaN,5.154683,7.339054,6.786583,12.510660,7.457284,4.580833,7.564839,NaN
2017-12,NaN,NaN,NaN,NaN,NaN,NaN,4.738191,7.816575,39.366019,48.135052,27.431400,23.518413,NaN
2018-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.135636,8.721228,12.365542,11.199767,4.699556,NaN
2018-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.156987,8.610000,4.942414,6.941026,NaN
